# Optimize Initial Conditions
## Yabox

In [16]:
from scipy.optimize import brute
import numpy as np
from datetime import datetime,timedelta
import pandas as pd
from yabox import DE

# Initialize Ray

In [17]:
from environs import Env
env = Env()
env.str("CUDA_DEVICE_ORDER",'PCI_BUS_ID')
env.int("CUDA_VISIBLE_DEVICES",1)
env.int("NUMBA_ENABLE_CUDASIM",1)
env.bool("OMPI_MCA_opal_cuda_support",True)

import os
import ray
MB=1024*1024
GB=MB*1024
ray.shutdown()
ray.init(object_store_memory=1*GB,memory=1*GB,
         #lru_evict=True,
         driver_object_store_memory=500*MB,num_gpus=4,num_cpus=10,
         ignore_reinit_error=True) # , include_webui=False, ignore_reinit_error=True)

@ray.remote(num_gpus=1)
def use_gpu():
    print("ray.get_gpu_ids(): {}".format(ray.get_gpu_ids()[0]))
    print("CUDA_VISIBLE_DEVICES: {}".format(os.environ["CUDA_VISIBLE_DEVICES"]))

use_gpu.remote()

2020-07-26 19:08:09,613	INFO resource_spec.py:212 -- Starting Ray with 0.98 GiB memory available for workers and up to 1.0 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-07-26 19:08:09,900	INFO services.py:1170 -- View the Ray dashboard at localhost:8267


ObjectID(b944ee5bb38dd1a5ffffffff010000c801000000)

# Load New and Process Data from website data.brasil.io

In [18]:
import get_data
LoadData=True

if LoadData:
    get_data.get_data()

Baixando arquivos brasil.io...
(pid=24442) ray.get_gpu_ids(): 3
(pid=24442) CUDA_VISIBLE_DEVICES: 3


# Functions to Load Processed Data

In [19]:
def load_confirmed(districtRegion, start_date):
    dateparse = lambda x: datetime.strptime(x, '%Y-%m-%d')
    df = pd.read_csv('./data/DRS_confirmados.csv',delimiter=',',parse_dates=True, date_parser=dateparse)
    y=[]
    x=[]
    for i in range(0,len(df.date)):
        y.append(df[districtRegion].values[i])
        x.append(df.date.values[i])
    df2=pd.DataFrame(data=y,index=x,columns=[""])
    df2=df2[start_date:]
    del x,y,df,dateparse
    return df2

def load_dead(districtRegion, start_date):
    dateparse = lambda x: datetime.strptime(x, '%Y-%m-%d')
    df = pd.read_csv('./data/DRS_mortes.csv',delimiter=',',parse_dates=True, date_parser=dateparse)
    y=[]
    x=[]
    for i in range(0,len(df.date)):
        y.append(df[districtRegion].values[i])
        x.append(df.date.values[i])
    df2=pd.DataFrame(data=y,index=x,columns=[""])
    df2=df2[start_date:]
    del x,y,df,dateparse
    return df2

# Load solver

In [20]:
import LearnerICRayNoLoadBH as L 

# Data for States

In [21]:
dfparam = pd.read_csv("data/param.csv")
dfparam
DRS=dfparam.DRS

In [22]:
dfSP = pd.read_csv("data/dados_municipios_SP.csv")

# lista DRSs
DRS = list(dfSP["DRS"].unique())
DRS.remove("Indefinido")

dfSP

,date,state,city,place_type,confirmed,deaths,order_for_place,is_last,popEst,city_ibge_code,confirmed_per_100k_inhabitants,death_rate,DRS
0,2020-07-25,SP,TOTAL,state,479481,21517,150,True,45919049.0,35.0,1044.18757,0.0449,Indefinido
1,2020-07-24,SP,TOTAL,state,463218,21206,149,False,45919049.0,35.0,1008.77089,0.0458,Indefinido
2,2020-07-23,SP,TOTAL,state,452007,20894,148,False,45919049.0,35.0,984.35619,0.0462,Indefinido
3,2020-07-22,SP,TOTAL,state,439446,20532,147,False,45919049.0,35.0,957.00153,0.0467,Indefinido
4,2020-07-21,SP,TOTAL,state,422669,20171,146,False,45919049.0,35.0,920.46549,0.0477,Indefinido
...,...,...,...,...,...,...,...,...,...,...,...,...,...
54126,2020-06-23,SP,Óleo,city,1,0,5,False,2496.0,3533809.0,40.06410,0.0000,DRS 09 - Marília
54127,2020-06-22,SP,Óleo,city,1,0,4,False,2496.0,3533809.0,40.06410,0.0000,DRS 09 - Marília
54128,2020-06-21,SP,Óleo,city,1,0,3,False,2496.0,3533809.0,40.06410,0.0000,DRS 09 - Marília
54129,2020-06-20,SP,Óleo,city,1,0,2,False,2496.0,3533809.0,40.06410,0.0000,DRS 09 - Marília


# Functions for Optimization

In [23]:
from scipy.integrate import odeint
import sys
import io
import gc

def create_f(districtRegion,e0,a0,r0,date, ratio, predict_range, version):
                
    def fobjective(point):
        
        dead=  load_dead(districtRegion,date)
        data = load_confirmed(districtRegion,date)*(1-ratio)-dead
        cleanRecovered=False
        s0, deltaDate, i0, d0, startNCases, weigthCases, weigthRecov = point
        end_date=datetime.strptime(date, "%Y-%m-%d") + timedelta(days=deltaDate)
        f=L.Learner.remote(districtRegion, end_date.strftime("%Y-%m-%d"), predict_range,\
                           s0, e0, a0, i0, r0, d0, startNCases, ratio, weigthCases, weigthRecov, \
                           cleanRecovered, version, data, dead, savedata=False)
        result = f.train.remote() 
        result = ray.get(result) 

        del end_date,cleanRecovered, data, dead, point,f         

        gc.collect()

        return result
    return fobjective

In [24]:
@ray.remote(memory=50 * 1024 * 1024, max_calls=1)
def opt(districtRegion,s0,i0,e0,a0,r0,d0,wcases,date,startNCases, ratio, predict_range, version):

    bounds=[(s0*0.8,s0*1.5),(-2,2),(i0/2,i0*2), (0,d0*1.2), (0,startNCases*1.2),\
              (0.3,0.8),(0.02,0.08)]
    maxiterations=500
    f=create_f(districtRegion,e0,a0,r0,date, ratio, predict_range, version)
    de = DE(f, bounds, maxiters=maxiterations)
    for step in de.geniterator():
        try:
            idx = step.best_idx
            norm_vector = step.population[idx]
            best_params = de.denormalize([norm_vector])
            del norm_vector, idx
        except:
            print("error in function evaluation")
    p=best_params[0]
    del f, bounds, data, dead,best_params
    
    return p

# Main Code

In [25]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines){
    return true;}

<IPython.core.display.Javascript object>

In [26]:
DRS=dfparam.DRS
display(DRS)
allDRS=True
version="105"
gc.enable()

optimal=[]
if allDRS:
    for districtRegion in DRS:
        #remove previous history file
        strFile='./results/history_'+districtRegion+version+'.csv'
        if os.path.isfile(strFile):
            os.remove(strFile)
        query = dfparam.query('DRS == "{}"'.format(districtRegion)).reset_index()
        parameters = np.array(query.iloc[:, 2:])[0]
        date,predict_range,s0,e0,a0,i0,r0,d0,startNCases,ratio,wcases,wrec = parameters
        optimal.append(opt.remote(districtRegion,s0,i0,e0,a0,r0,d0,wcases,date,startNCases, ratio, predict_range, version))        
else:
    districtRegion = "DRS 01 - Grande São Paulo" #'DRS 08 - Franca' \
    #'DRS 14 - São João da Boa Vista' #'DRS 04 - Baixada Santista' \
    #'DRS 11 - Presidente Prudente' #'DRS 13 - Ribeirão Preto' \
    #'DRS 05 - Barretos' #'DRS 12 - Registro' #'DRS 15 - São José do Rio Preto' \
    #'DRS 10 - Piracicaba'#'DRS 17 - Taubaté'#'DRS 02 - Araçatuba'# \
    #'DRS 03 - Araraquara' #DRS 07 - Campinas'#'DRS 16 - Sorocaba'#'DRS 06 - Bauru' \
    #'DRS 09 - Marília' #"DRS 01 - Grande São Paulo"

    #remove previous history file
    strFile='./results/history_'+districtRegion+version+'.csv'
    if os.path.isfile(strFile):
        os.remove(strFile)
    query = dfparam.query('DRS == "{}"'.format(districtRegion)).reset_index()
    parameters = np.array(query.iloc[:, 2:])[0]
    start_date,predict_range,s0,e0,a0,i0,r0,d0,startNCases,ratio,wcases,wrec = parameters
    optimal.append(opt.remote(districtRegion,s0,i0,e0,a0,r0,d0,wcasesdate,startNCases, ratio, predict_range, version))        

0          DRS 01 - Grande São Paulo
1                 DRS 02 - Araçatuba
2                DRS 03 - Araraquara
3          DRS 04 - Baixada Santista
4                  DRS 05 - Barretos
5                     DRS 06 - Bauru
6                  DRS 07 - Campinas
7                    DRS 08 - Franca
8                   DRS 09 - Marília
9                DRS 10 - Piracicaba
10      DRS 11 - Presidente Prudente
11                 DRS 12 - Registro
12           DRS 13 - Ribeirão Preto
13    DRS 14 - São João da Boa Vista
14    DRS 15 - São José do Rio Preto
15                 DRS 16 - Sorocaba
16                  DRS 17 - Taubaté
Name: DRS, dtype: object

In [ ]:
optimal=ray.get(optimal)


2020-07-26 19:08:31,635	WARNING worker.py:1090 -- WARNING: 30 PYTHON workers have been started. This could be a result of using a large number of actors, or it could be a consequence of using nested tasks (see https://github.com/ray-project/ray/issues/3644) for some a discussion of workarounds.
2020-07-26 19:08:31,933	WARNING worker.py:1090 -- WARNING: 31 PYTHON workers have been started. This could be a result of using a large number of actors, or it could be a consequence of using nested tasks (see https://github.com/ray-project/ray/issues/3644) for some a discussion of workarounds.
2020-07-26 19:08:31,935	WARNING worker.py:1090 -- WARNING: 32 PYTHON workers have been started. This could be a result of using a large number of actors, or it could be a consequence of using nested tasks (see https://github.com/ray-project/ray/issues/3644) for some a discussion of workarounds.
2020-07-26 19:08:31,936	WARNING worker.py:1090 -- WARNING: 33 PYTHON workers have been started. This could be a 

(pid=24605) basinhopping step 0: f 8.67461e+07
(pid=24603) basinhopping step 0: f 1.34951e+09
(pid=24761) basinhopping step 0: f 1.74063e+07
(pid=24606) basinhopping step 0: f 5.54531e+06
(pid=24804) basinhopping step 0: f 1.33417e+07
(pid=24601) basinhopping step 0: f 1.42434e+06
(pid=24603) basinhopping step 1: f 1.34951e+09 trial_f 1.32744e+11 accepted 0  lowest_f 1.34951e+09
(pid=24601) basinhopping step 1: f 1.42434e+06 trial_f 1.67024e+06 accepted 0  lowest_f 1.42434e+06
(pid=24604) basinhopping step 0: f 6.98372e+07
(pid=24607) basinhopping step 0: f 1.44329e+06
(pid=24604) basinhopping step 1: f 6.98372e+07 trial_f 7.9038e+07 accepted 0  lowest_f 6.98372e+07
(pid=24602) basinhopping step 0: f 7.32487e+06
(pid=24783) basinhopping step 0: f 188228
(pid=24602) basinhopping step 1: f 7.32487e+06 trial_f 9.74728e+06 accepted 0  lowest_f 7.32487e+06
(pid=24783) basinhopping step 1: f 188228 trial_f 242264 accepted 0  lowest_f 188228
(pid=24783) basinhopping step 2: f 188228 trial_f 2

(pid=24604) basinhopping step 4: f 6.98372e+07 trial_f 7.89914e+07 accepted 0  lowest_f 6.98372e+07
(pid=24605) basinhopping step 4: f 8.67461e+07 trial_f 1.00296e+08 accepted 0  lowest_f 8.67461e+07
(pid=24783) basinhopping step 8: f -1.74305e+06 trial_f 242182 accepted 0  lowest_f -1.74305e+06
(pid=24606) basinhopping step 4: f -2.16249e+08 trial_f 9.06994e+06 accepted 0  lowest_f -2.16249e+08
(pid=24783) basinhopping step 9: f -1.74305e+06 trial_f 242264 accepted 0  lowest_f -1.74305e+06
(pid=24761) basinhopping step 5: f 1.6641e+07 trial_f 1.7815e+07 accepted 0  lowest_f 1.6641e+07
(pid=24605) basinhopping step 5: f 8.67461e+07 trial_f 1.00881e+08 accepted 0  lowest_f 8.67461e+07
(pid=24604) basinhopping step 5: f 6.98372e+07 trial_f 7.89875e+07 accepted 0  lowest_f 6.98372e+07
(pid=24609) basinhopping step 2: f 787506 trial_f 787506 accepted 1  lowest_f 787506
(pid=24609) found new global minimum on step 2 with function value 787506
(pid=24602) basinhopping step 5: f -7.62967e+07 

2020-07-26 19:09:41,912	WARNING worker.py:1090 -- WARNING: 34 PYTHON workers have been started. This could be a result of using a large number of actors, or it could be a consequence of using nested tasks (see https://github.com/ray-project/ray/issues/3644) for some a discussion of workarounds.
2020-07-26 19:09:41,914	WARNING worker.py:1090 -- WARNING: 35 PYTHON workers have been started. This could be a result of using a large number of actors, or it could be a consequence of using nested tasks (see https://github.com/ray-project/ray/issues/3644) for some a discussion of workarounds.


(pid=24757) basinhopping step 10: f -5.90717e+06 trial_f 405451 accepted 0  lowest_f -5.90717e+06


...

(pid=59443) basinhopping step 10: f -1.31873e+08 trial_f 1.3912e+07 accepted 0  lowest_f -1.31873e+08
(pid=59646) basinhopping step 2: f 4.62013e+06 trial_f 6.92517e+06 accepted 0  lowest_f 4.62013e+06
(pid=59388) basinhopping step 10: f -725873 trial_f 314671 accepted 0  lowest_f -725873
(pid=59559) basinhopping step 3: f -3.38079e+12 trial_f 9.06685e+10 accepted 0  lowest_f -3.38079e+12


2020-07-26 21:49:39,800	WARNING worker.py:1090 -- WARNING: 34 PYTHON workers have been started. This could be a result of using a large number of actors, or it could be a consequence of using nested tasks (see https://github.com/ray-project/ray/issues/3644) for some a discussion of workarounds.
2020-07-26 21:49:39,803	WARNING worker.py:1090 -- WARNING: 35 PYTHON workers have been started. This could be a result of using a large number of actors, or it could be a consequence of using nested tasks (see https://github.com/ray-project/ray/issues/3644) for some a discussion of workarounds.


(pid=59591) basinhopping step 6: f 1.20478e+07 trial_f 1.60703e+07 accepted 0  lowest_f 1.20478e+07
(pid=59673) basinhopping step 0: f 656076
(pid=59645) basinhopping step 0: f 662470
(pid=59590) basinhopping step 2: f 1.85443e+07 trial_f 1.85443e+07 accepted 1  lowest_f 1.85443e+07
(pid=59590) found new global minimum on step 2 with function value 1.85443e+07
(pid=59645) basinhopping step 1: f 662470 trial_f 940704 accepted 0  lowest_f 662470
(pid=59645) basinhopping step 2: f 662470 trial_f 940704 accepted 0  lowest_f 662470
(pid=59672) basinhopping step 0: f 1.35815e+07
(pid=59727) basinhopping step 0: f 7.57586e+06
(pid=59755) basinhopping step 0: f 462301
(pid=59618) basinhopping step 3: f 1.03969e+06 trial_f 1.03969e+06 accepted 1  lowest_f 1.03969e+06
(pid=59618) found new global minimum on step 3 with function value 1.03969e+06
(pid=59755) basinhopping step 1: f 462301 trial_f 535360 accepted 0  lowest_f 462301
(pid=59700) basinhopping step 0: f 1.33812e+07
(pid=59527) basinhop

2020-07-26 21:50:07,256	WARNING worker.py:1090 -- WARNING: 34 PYTHON workers have been started. This could be a result of using a large number of actors, or it could be a consequence of using nested tasks (see https://github.com/ray-project/ray/issues/3644) for some a discussion of workarounds.
2020-07-26 21:50:07,260	WARNING worker.py:1090 -- WARNING: 35 PYTHON workers have been started. This could be a result of using a large number of actors, or it could be a consequence of using nested tasks (see https://github.com/ray-project/ray/issues/3644) for some a discussion of workarounds.


(pid=59646) basinhopping step 5: f -1.07965e+08 trial_f 6.4245e+06 accepted 0  lowest_f -1.07965e+08
(pid=59619) basinhopping step 6: f 1.29103e+08 trial_f 1.61548e+08 accepted 0  lowest_f 1.29103e+08


In [ ]:
for i in range(0,len(DRS)):    

    districtRegion=DRS[i]
    query = dfparam.query('DRS == "{}"'.format(districtRegion)).reset_index()
    parameters = np.array(query.iloc[:, 2:])[0]
    startdate,predict_range,s0,e0,a0,i0,r0,d0,startNCases,ratio,wcases,wrec = parameters
    Date = datetime.strptime(startdate, "%Y-%m-%d")
    end_date = Date + timedelta(days=+int(optimal[i][1]))
    dateStr=end_date.strftime("%Y-%m-%d")

    j = query['index'].values[0]
    dfparam.at[j, "s0"] = optimal[i][0]
    dfparam.at[j, "start-date"] = dateStr
    dfparam.at[j, "i0"] = optimal[i][2]
    dfparam.at[j, "WCASES"] = optimal[i][3]
    dfparam.at[j, "WREC"] = optimal[i][4]

    dfparam.to_csv("data/param_optimized_Yabox.csv", sep=",", index=False)
    dfparam
    